In [25]:
# Add Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense,Dropout,Input # type: ignore
from tensorflow.keras.callbacks import EarlyStopping # type: ignore


In [ ]:
# Load the dataset
data1 = pd.read_csv('../data/bank-additional-full_normalised.csv')

In [26]:
# Load the dataset
data = pd.read_csv('../data/bank-additional-full_normalised.csv')
# Check if all values are between 0 and 1
if not ((data >= 0) & (data <= 1)).all().all():
    print("Values outside the range [0, 1] found in the dataset.")
    print(data[(data < 0) | (data > 1)])

In [70]:
# Separate features and target
X = data.drop('class', axis=1)
y = data['class'].to_numpy()

In [71]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Define the DNN model
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])


# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
# class_weights_dict = {cls: weight for cls, weight in zip(np.unique(y_train), class_weights)}
class_weights_dict  = {0 : class_weights[0], 1 : class_weights[1]};
print(class_weights_dict)

In [ ]:
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model

history = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=100,
    validation_split=0.2,
    callbacks=[early_stopping],
    class_weight=class_weights_dict
)


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

In [ ]:
from sklearn.metrics import classification_report

# Evaluate the model


# Get predictions
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)  # Convert probabilities to binary classes

# Generate classification report
report = classification_report(y_test, y_pred_classes)
print(report)